### myTraidingBot
"""
Script for training Stock Trading Bot.

Usage:
  train.py <train-stock> <val-stock> [--strategy=<strategy>]
    [--window-size=<window-size>] [--batch-size=<batch-size>]
    [--episode-count=<episode-count>] [--model-name=<model-name>]
    [--pretrained] [--debug]

Options:
  --strategy=<strategy>             Q-learning strategy to use for training the network. Options:
                                      `dqn` i.e. Vanilla DQN,
                                      `t-dqn` i.e. DQN with fixed target distribution,
                                      `double-dqn` i.e. DQN with separate network for value estimation. [default: t-dqn]
  --window-size=<window-size>       Size of the n-day window stock data representation
                                    used as the feature vector. [default: 10]
  --batch-size=<batch-size>         Number of samples to train on in one mini-batch
                                    during training. [default: 32]
  --episode-count=<episode-count>   Number of trading episodes to use for training. [default: 50]
  --model-name=<model-name>         Name of the pretrained model to use. [default: model_debug]
  --pretrained                      Specifies whether to continue training a previously
                                    trained model (reads `model-name`).
  --debug                           Specifies whether to use verbose logs during eval operation.
"""

In [ ]:
#cd C:\Users\Иван\tf\myTradingBot_withMemory

In [ ]:
import cProfile
import re
cProfile.run('re.compile("foo|bar")')

In [ ]:
import logging
import coloredlogs

from docopt import docopt

from trading_bot.agent import Agent
from trading_bot.methods import train_model, evaluate_model
from trading_bot.utils import (
    get_stock_data,
    format_currency,
    format_position,
    show_train_result,
    switch_k_backend_device
)


In [ ]:
switch_k_backend_device()

In [ ]:
# Init

# Train data
train_stock = r'D:\share\finam\data\GAZP\hourly\GAZP_2022.csv'
# Validate data
val_stock = r'D:\share\finam\data\GAZP\hourly\GAZP_2022.csv'
window_size = 10
batch_size = 32
# Qty training episodes
ep_count = 10 
strategy= "t-dqn"#'double-dqn' #"t-dqn"
# Model file name
model_name="model_M_t-dqn_tst"
# Create new model or not
pretrained=False
debug=True

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

In [ ]:
agent = Agent(window_size, strategy=strategy, pretrained=pretrained, model_name=model_name)


In [ ]:
train_data = get_stock_data(train_stock,tfCounts=100000)


In [ ]:
val_data = get_stock_data(val_stock)

In [ ]:
train_data = train_data[0:100]
val_data = val_data[0:100]

In [ ]:
initial_offset = val_data[1] - val_data[0]

### Train

In [ ]:
for episode in range(ep_count + 1):
    #cProfile.run('train_result = train_model(agent, episode, train_data, ep_count=ep_count,batch_size=batch_size, window_size=window_size)')
    train_result = train_model(agent, episode, train_data, ep_count=ep_count,batch_size=batch_size, window_size=window_size)
    val_result, _ = evaluate_model(agent, val_data, window_size, debug)
    show_train_result(train_result, val_result, initial_offset)


In [ ]:
val_result, _ = evaluate_model(agent, val_data, window_size, True)

In [ ]:
show_train_result(train_result, val_result, initial_offset)

In [ ]:
agent.target_model.predict(get_state(train_data,100,11))

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import pandas as pd
from trading_bot.ops import (
    get_state
)

state = [agent.act(get_state(train_data,i,11))*10+250 for i in  range(10,60)]
# print(agent.act(state))
df = train_data.copy()
#df[] = pd.Series(state)
plt.plot(df[10:60])
plt.scatter(list(range(len(state))),state)


In [ ]:
agent.model.save('Test_Model.h5')

In [ ]:
from keras.models import load_model, clone_model
agent.model = load_model('Test_Model.h5')

In [ ]:
val_data[:10]
